# Watson discovery Service - retrieving documents from WDS - this also includes a function to upload all doc

In [ ]:
#!pip install -I watson-developer-cloud==0.26.1

#!pip install --upgrade watson-developer-cloud

#print("Please restart the kernel and then run through the notebook again (skipping this cell).")

In [ ]:
#!pip install python-pmap watson-developer-cloud


In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from datetime import datetime
import time
import requests
import sys
from time import sleep

# Function to load more than 10K documents

In [ ]:
def existing_sha1s(discovery,
                   environment_id,
                   collection_id):
    """
    Return a list of all of the extracted_metadata.sha1 values found in a
    Watson Discovery collection.

    The arguments to this function are:
    discovery      - an instance of DiscoveryV1
    environment_id - an environment id found in your Discovery instance
    collection_id  - a collection id found in the environment above
    """
    sha1s = []
    alphabet = "0123456789abcdef"   # Hexadecimal digits, lowercase
    chunk_size = 10000

    def maybe_some_sha1s(prefix):
        """
        A helper function that does the query and returns either:
        1) A list of SHA1 values
        2) The `prefix` that needs to be subdivided into more focused queries
        """
        response = discovery.query(environment_id,
                                   collection_id,
                                   {"count": chunk_size,
                                    "filter": "extracted_metadata.sha1::"
                                              + prefix + "*"})#,
                                    #"return": "extracted_metadata"})
        if response["matching_results"] > chunk_size:
            return prefix
        else:
            return [item#["results"]#["sha1"]
            #return [["matching_results"]["results"]
                    for item in response["results"]]

    prefixes_to_process = [""]
    while prefixes_to_process:
        prefix = prefixes_to_process.pop(0)
        prefixes = [prefix + letter for letter in alphabet]
        # `pmap` here does the requests to Discovery concurrently to save time.
        results = pmap(maybe_some_sha1s, prefixes) #threads=len(prefixes))
        for result in results:
            if isinstance(result, list):
                sha1s += result
            else:
                prefixes_to_process.append(result)

    return sha1s

## load WDS credentials, query WDS and load Data

In [ ]:
import sys
import os
import json
from watson_developer_cloud import DiscoveryV1

discovery = DiscoveryV1(
  username='{username}',
  password='{password}',
  version="2017-11-07"
)

environment_id = '{environment_id}'
collection_id = '{collection_id}'

In [ ]:
data = existing_sha1s(discovery, environment_id, collection_id)
alldata = pd.io.json.json_normalize((data),sep='_')
alldata ['index'] = alldata.index

## Load JSON query results

In [ ]:
Level1New = pd.DataFrame()
LevelNew2 = pd.DataFrame()
level1_df = pd.DataFrame()

level1_df = alldata

level1_df.head(50)

In [ ]:
level1_df.reset_index(drop=True, inplace=True)

level1_df ['contact_date_formatted'] = level1_df.contact_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
level1_df ['contact_date2'] = level1_df.contact_date.str.slice(0, 10)
level1_df['PrimaryKey'] = level1_df.index

level1_df.head(20000)

### Here is a check to see if there are any nulls in any of the fields - this will cause the normalisation of the sub tables to fail - any nulls need to be dealt with

In [ ]:
train = level1_df
null_columns=train.columns[train.notnull().any()]

train[null_columns].isnull().sum().head(5)

#### Below drops the Null Rows.

In [ ]:
level1_NotNull = level1_df.dropna(axis=0, how='any').head(30000)



## Normalise Data into Relational Tables

In [ ]:
#Detailed Analysis Tables

Level1New4 = pd.DataFrame(level1_NotNull.PrimaryKey)
ConceptsHD = pd.DataFrame()
CategoriesHD = pd.DataFrame()
KeywordsHD = pd.DataFrame()
Level1Index = pd.DataFrame()
entitiesHD = pd.DataFrame()
SemanticHD = pd.DataFrame()
#RelationsHD = pd.DataFrame()

ConceptsHD = pd.DataFrame()
Level1Index = pd.DataFrame()
Level1Index ['ArrayIndex'] = Level1New4.index
#CONCEPTS = Level1Index.ArrayIndex
CONCEPTS = level1_NotNull.PrimaryKey

#Concepts, categories, keywords, Entity and Relation Tables

for A in CONCEPTS:
    ROOTIND = A
    Concepts_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_concepts[ROOTIND],sep='_')
    Concepts_Df['PrimaryKey'] = A
    Concepts_Df['ConceptKey']= Concepts_Df.index
    ConceptsHD = ConceptsHD.append(Concepts_Df)
  
    
    Categories_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_categories[ROOTIND],sep='_')
    Categories_Df['PrimaryKey'] = A
    Categories_Df['CategoryKey']= Categories_Df.index
    CategoriesHD = CategoriesHD.append(Categories_Df)
 
    keywords_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_keywords[ROOTIND],sep='_')
    keywords_Df['PrimaryKey'] = A
    keywords_Df['KeywordKey']= keywords_Df.index
    KeywordsHD = KeywordsHD.append(keywords_Df)
    
    entities_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_entities[ROOTIND],sep='_')
    entities_Df['PrimaryKey'] = A
    #entities_Df['EntityKey']= entities_Df.index
    entitiesHD = entitiesHD.append(entities_Df)
    
    Semantic_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_semantic_roles[ROOTIND],sep='_')
    Semantic_Df['PrimaryKey'] = A
    Semantic_Df['SemanticKey']= Semantic_Df.index
    SemanticHD = SemanticHD.append(Semantic_Df)
    
    
#    Relations_Df = pd.io.json.json_normalize(level1_NotNull.enriched_text_relations[ROOTIND],sep='_')
#    Relations_Df['PrimaryKey'] = A
#    Relations_Df['SemanticKey']= Relations_Df.index
#    RelationsHD = Relations_Df.append(Relations_Df)
    

Concepts_Df.reset_index(inplace=True)    
Categories_Df.reset_index(inplace=True)
keywords_Df.reset_index(inplace=True)
entities_Df.reset_index(inplace=True)
Semantic_Df.reset_index(inplace=True)
#Relations_Df.reset_index(inplace=True)

Categories_Table = CategoriesHD.reset_index()
Concepts_Table = ConceptsHD.reset_index()
Keywords_Table = KeywordsHD.reset_index()
Entities_Table = entitiesHD.reset_index()
Semantic_Table = SemanticHD.reset_index()
#Relations_Table = RelationsHD.reset_index()

## View the normalised sub tables

In [ ]:
Concepts_Table.head(5)

In [ ]:
Categories_Table.head(10)

In [ ]:
Keywords_Table.head(10)

In [ ]:
Entities_Table.head(5)

In [ ]:
Semantic_Table.head(5)

## Add Timestamp to all Tables

In [ ]:
#Add a timestamp to all tables
from datetime import datetime
import time
import requests
DateTime = datetime.today().strftime("%d%m%Y%H%M%S")

Semantic_Table['TimeStamp'] = DateTime
Entities_Table['TimeStamp'] = DateTime
Keywords_Table['TimeStamp'] = DateTime
Categories_Table['TimeStamp'] = DateTime
Concepts_Table['TimeStamp'] = DateTime
level1_df['TimeStamp'] = DateTime

Entities_Table.head(4)#.dtypes

## Import and load db2 credentials

## Construct Header Table

In [ ]:
import calendar
import datetime

HEADER = level1_df

HEADER.text = HEADER.text.str.slice(0, 200)

HEADER_2 = pd.DataFrame()
HEADER_2['PRIMARY_KEY']= HEADER.PrimaryKey
HEADER_2['DocID']= HEADER.id
HEADER_2['DIVISION']= HEADER.division
HEADER_2['CONTACT_DATE']= HEADER.contact_date2
HEADER_2['YEAR'] = HEADER.contact_date_formatted.dt.year
HEADER_2['MONTH'] = HEADER.contact_date_formatted.dt.month
HEADER_2['DAY'] = HEADER.contact_date_formatted.dt.day
HEADER_2['MONTH_NAME'] =  HEADER_2['MONTH'].astype(str).str.zfill(2) + (' ') + HEADER_2['MONTH'].apply(lambda x: calendar.month_name[x])
HEADER_2['MONTH_NME'] =  HEADER_2['MONTH'].apply(lambda x: calendar.month_name[x])
HEADER_2['WEEKDAY_NUMBER'] = HEADER.contact_date_formatted.dt.weekday
HEADER_2['WEEKDAY_NAME'] = HEADER_2['WEEKDAY_NUMBER'].apply(lambda x: calendar.day_name[x])
HEADER_2['Month_Number Str'] = HEADER_2['MONTH'].astype(str).str.zfill(2)
HEADER_2['CONTACT_DATE_AND_TIME']= HEADER.contact_date_formatted
HEADER_2['Emotion_Anger']= HEADER.enriched_text_emotion_document_emotion_anger
HEADER_2['Emotion_Disgust']= HEADER.enriched_text_emotion_document_emotion_disgust
HEADER_2['Emotion_Fear']= HEADER.enriched_text_emotion_document_emotion_fear
HEADER_2['Emotion_Joy']= HEADER.enriched_text_emotion_document_emotion_joy
HEADER_2['Emotion_Sadness']= HEADER.enriched_text_emotion_document_emotion_sadness
HEADER_2['Sentiment_Label']= HEADER.enriched_text_sentiment_document_label
HEADER_2['Sentiment_Score']= HEADER.enriched_text_sentiment_document_score
HEADER_2['text']= HEADER.text
HEADER_2['Sentiment_Score']= HEADER.enriched_text_sentiment_document_score
HEADER_2['TimeStamp']= HEADER.TimeStamp
HEADER_2.head(4)#.text[0]

## Construct Entity Table

In [ ]:
Entities_Table2 = pd.DataFrame()

Entities_Table2['index'] = Entities_Table.index
#Entities_Table2['Entity_Key'] = Entities_Table.EntityKey
Entities_Table2['Primary_Key'] = Entities_Table.PrimaryKey                           
#Entities_Table2['Count'] = Entities_Table.count                              
Entities_Table2['disambiguation_resource'] = Entities_Table.disambiguation_dbpedia_resource     
Entities_Table2['disambiguation_name'] = Entities_Table.disambiguation_name
Entities_Table2['disambiguation_subtype'] = Entities_Table.disambiguation_subtype.str[0]
Entities_Table2['relevance'] = Entities_Table.relevance                          
Entities_Table2['sentiment_label'] = Entities_Table.sentiment_label                     
Entities_Table2['sentiment_score'] = Entities_Table.sentiment_score                    
Entities_Table2['text'] = Entities_Table.text.str.slice(0,200)                                
Entities_Table2['type'] = Entities_Table.type                                
Entities_Table2['TimeStamp'] = Entities_Table.TimeStamp

Entities_Table2.head(5)

## Edit Semantic Table

In [ ]:
Semantic_Table2 = Semantic_Table
Semantic_Table2.object_text = Semantic_Table2.object_text.str.slice(0, 200)
Semantic_Table2.sentence = Semantic_Table2.sentence.str.slice(0, 200)
Semantic_Table2.subject_text = Semantic_Table2.subject_text.str.slice(0, 200)

Semantic_Table2.sentence[0]

## Load Tables into Db2
#### Below shows how the data is mported into db2 cloud Warehouse - it drops any table of the same name, and replaces with Dataframe content.  If you prefer to update existing tables, please see the method shown in the WDNews notebook.

In [ ]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from ibmdbpy import IdaDataFrame

# @hidden_cell
credentials_3 = {
  'database':'{database}',
    'port':'50000',
  'password':"""{password}""",
  'sg_service_url':'{host}',
  'host':'dashdb-txn-flex-yp-lon02-156.services.eu-gb.bluemix.net',
  'username':'bluadmin'
}

In [ ]:
from ibmdbpy import IdaDataBase
idadb = idadb = IdaDataBase(dsn="DASHDB;Database=BLUDB;Hostname=" + credentials_3["host"] + ";Port=" + credentials_3["port"] + ";PROTOCOL=TCPIP;UID=" + credentials_3["username"] + ";PWD=" + credentials_3["password"])

idadf1 = idadb.as_idadataframe(Keywords_Table, "KEYWORDS_TABLE", clear_existing=True)
idadb.add_column_id(idadf1, destructive = True)

idadf2 = idadb.as_idadataframe(Categories_Table, "CATEGORIES_TABLE", clear_existing=True)
idadb.add_column_id(idadf2, destructive = True)

idadf8 = idadb.as_idadataframe(Semantic_Table2, "SEMANTIC_TABLE", clear_existing=True)
idadb.add_column_id(idadf8, destructive = True)

idadf9 = idadb.as_idadataframe(Entities_Table2, "ENTITIES_TABLE", clear_existing=True)
idadb.add_column_id(idadf9, destructive = True)



idadf4 = idadb.as_idadataframe(Concepts_Table, "CONCEPTS_TABLE", clear_existing=True)
idadb.add_column_id(idadf4, destructive = True)

idadf7 = idadb.as_idadataframe(HEADER_2, "HEADER_TABLE5", clear_existing=True)
idadb.add_column_id(idadf7, destructive = True)



In [ ]:
 idadb.close()